In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.layers import Input, Dense, Activation, TextVectorization, Embedding, GlobalAveragePooling1D
from keras.models import Model
import tensorflow as tf

# Vetorização e Embedding de textos

# Exercício 1
*Objetivo: entender como funciona a camada TextVectorization*

Quando usamos sklearn, temos os objetos do tipo *vectorizer* que transformam strings em matrizes, em formatos que são "compreensíveis" por uma máquina. Em Keras, vamos usar a camada do tipo `TextVectorization` para o mesmo fim.

No código abaixo:

1. O que significa `output_mode='multi_hot'`?
1. Como acessamos o vocabulário do vectorizer?
1. O que significa o valor em `y[0,2]`?
1. O que significa a palavra `[UNK]` no vocabulário do vectorizer?
1. Da forma que está, o vetorizador é equivalente a qual construção do sklearn?

In [ ]:
toy_dataset = ["este é um texto", "outro texto este é", "outro texto só é peixe na virada da maré"]

text_vectorization = TextVectorization(output_mode='multi_hot')
text_vectorization.adapt(toy_dataset)
y = text_vectorization(toy_dataset)
print(y)
print(text_vectorization.get_vocabulary())

# Exercício 2
*Objetivo: usar um vetorizador dentro do modelo de rede neural*

O código abaixo incorpora o vetorizador ao modelo da rede neural de forma semelhante que o CountVectorizer e o LogisticRegression se acoplavam em uma pipeline.

Em Keras, o vetorizador deve ser treinado em separado do restante da rede porque o treinamento do vetorizador é feito na CPU, ao passo que o restante da rede pode ser treinado na GPU.

Outro detalhe importante é que a conversão dos rótulos para one-hot encoding deve ser feita fora do modelo (os modelos do Sklearn fazem essa operação internamente).

No código abaixo:

1. Aumente o número de épocas de treinamento para 30 e verifique o accuracy no conjunto de teste
1. Modifique o `output_mode` do vetorizador para `count` e depois para `tf_idf`. Houve modificação no accuracy? 

In [ ]:
df = pd.read_csv('datasets/IMDB Dataset.csv')
ohe = OneHotEncoder()
y_ohe = ohe.fit_transform(df['sentiment'].to_numpy().reshape((-1,1))).todense()
X_train, X_test, y_train, y_test = train_test_split(df['review'], y_ohe)

In [ ]:
vocab_size = 1000
def multihot_softmax_model(vectorize_layer, vocab_size=vocab_size):
    input_layer = Input(shape=(1,), dtype=tf.string)
    x = input_layer
    x = vectorize_layer(x)
    x = Dense(2, name='classificador')(x)
    x = Activation('softmax')(x)
    return Model(input_layer, x)

vectorize_layer = TextVectorization(output_mode='multi_hot', max_tokens=vocab_size, pad_to_max_tokens=True)
vectorize_layer.adapt(X_train)
clf = multihot_softmax_model(vectorize_layer)
print(clf.summary())
clf.compile(loss='categorical_crossentropy', metrics=['accuracy'])
history = clf.fit(X_train, y_train, epochs=30, verbose=1) # validation_split=0.1
clf.evaluate(X_test, y_test)

# Exercício 3
*Objetivo: usar embeddings para representar documentos*

Até o momento, estamos representando nossos documentos como sequências de *inteiros*, em que cada inteiro representa uma palavra:

In [ ]:
toy_dataset = ["este é um texto", "outro texto este é", "outro texto só é peixe na virada da maré"]

text_vectorization = TextVectorization(output_mode='int', max_tokens=vocab_size, pad_to_max_tokens=True, output_sequence_length=10)
text_vectorization.adapt(toy_dataset)
y = text_vectorization(toy_dataset)
print(y)
print(text_vectorization.get_vocabulary())

Experimente colocar frases com palavras desconhecidas no trecho abaixo:

In [ ]:
print(text_vectorization(["este texto tem um monte de palavras desconhecidas e isso realmente confunde nosso vetorizador"]))

Uma vantagem da representação como sequência de inteiros é que preservamos a ordem das palavras. Uma desvantagem é que essa representação não é tão fácil de classificar como as que estávamos usando até agora.

Uma solução para isso é mapear os inteiros para vetores - esses sim, num espaço vetorial fácil de classificar. Para isso, usamos uma camada chamada Embedding, que funciona exatamente como um dicionário:

In [ ]:
emb = Embedding(vocab_size, 2)
y = text_vectorization(["texto de exemplo"])
print(emb(y))

Por fim, temos mais um problema: a camada de embedding nos dá um tensor de dimensão: (batch x tempo x dimensão do embedding). Porém, nossa rede neural só consegue classificar elementos de dimensão (batch x features). Uma solução que podemos adotar para isso é tirar a média ao longo do tempo de todos os vetores do embedding, usando a camada GlobalAveragePooling1D:

In [ ]:
avg = GlobalAveragePooling1D()
print(avg(emb(y)))

Mas, você deve estar se perguntando: isso não vai fazer com que a informação de tempo seja jogada fora?

Sim. Por enquanto, e só porque não sabemos lidar com o tempo *ainda*. Vamos usar a técnica atual para fazer um classificador e investigá-lo.

---

No código abaixo, encontre:

1. Qual é a dimensão do embedding realizado.
1. Quantos parâmetros existem na camada de classificação.
1. Qual é o desempenho da rede após o treinamento?
1. O que acontece se o parâmetro `output_sequence_length` for reduzido no vectorize_layer? 

In [ ]:
def avg_embedding_softmax_model(vectorize_layer, vocab_size=vocab_size):
    input_layer = Input(shape=(1,), dtype=tf.string)
    x = input_layer
    x = vectorize_layer(x)
    x = Embedding(vocab_size, 2, name='projecao')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(2, name='classificador')(x)
    x = Activation('softmax')(x)
    return Model(input_layer, x)

vectorize_layer = TextVectorization(output_mode='int', max_tokens=vocab_size, pad_to_max_tokens=True, output_sequence_length=200)
vectorize_layer.adapt(X_train)
clf = avg_embedding_softmax_model(vectorize_layer)
print(clf.summary())
clf.compile(loss='categorical_crossentropy', metrics=['accuracy'])
history = clf.fit(X_train, y_train, epochs=60, verbose=1) # validation_split=0.1
clf.evaluate(X_test, y_test)

## Exercício 4
*Objetivo: visualizar e analisar as dimensões de embeddings atingidas pelo classificador*

As projeções abaixo mostram a projeção (embedding) de cada palavra conforme foram atingidas pelo classificador.

1. Analisando as figuras, você diria que aumentar o número de dimensões do embedding poderia levar a um ganho de desempenho no classificador?
1. Analisando as figuras, você diria que é possível reduzir o número de dimensões do embedding para tornar nosso classificador ainda menor?

In [ ]:
import plotly.express as px

# Visualização: onde foi parar cada palavra?
projecoes = clf.get_layer('projecao').get_weights()[0]
vocabulario = vectorize_layer.get_vocabulary()
y_pred_ohe = clf.predict(vocabulario)
y_pred = ohe.inverse_transform(y_pred_ohe)

df = pd.DataFrame()
df['dim_1'] = projecoes[:,0]
df['dim_2'] = projecoes[:,1]
df['word'] = vocabulario
df['prediction'] = y_pred

px.scatter(df, x="dim_1", y="dim_2", color="prediction", hover_data=["word"], title="Onde foi cada palavra?", width=600, height=600)

In [ ]:
import plotly.express as px
df = pd.read_csv('embeddings_over_epochs.csv') # Gravei esse arquivo antecipadamente - demora muito para refazê-lo!
df.head()
df['prediction'][0] = 'positive' # Isso é definitivamente um hack. Sem isso, o plotly não vê o label 'positive' na primeira época, e começa a remover o label 'positive' dos plots subsequentes.

px.scatter(df, x="dim_1", y="dim_2", animation_frame="epoch", animation_group="word",
            color="prediction", hover_name="word", title="Training a 2D word embedding for sentiment analysis <br><sup>Where did each word go in the embedding space?</sup>",
          range_x=[-15,15], range_y=[-15,15],
          width=800, height=800
          )


# Exercício 5
*Objetivo: fazer um classificador de intenções*

O código abaixo carrega um dataset de frases e intenções voltado para fazer um chatbot. Depois disso, ele seleciona as N_INTENTS intenções com mais exemplos na base de dados, simplificando o problema de classificação.

Com base nesses dados e no que já vimos hoje:

1. Programe, treine e avalie um classificador de intenções para o chatbot usando a topologia que vimos hoje. No treinamento, aumente o número de épocas até que o erro no conjunto de *validação* fique estável.
1. Faça um plot do espaço de embeddings e verifique como as palavras se organizam nele.

In [ ]:
TRAIN_URL = 'https://raw.githubusercontent.com/AmFamMLTeam/ACID/master/customer_training.csv'
TEST_URL = 'https://raw.githubusercontent.com/AmFamMLTeam/ACID/master/customer_testing.csv'
df_train = pd.read_csv(TRAIN_URL)
df_test = pd.read_csv(TEST_URL)
intent_count = df_train['INTENT_NAME'].value_counts()
N_INTENTS = 3
allowed_intents = intent_count[0:N_INTENTS].index
filter_train = df_train['INTENT_NAME'].isin(allowed_intents)
df_train_ = df_train[filter_train]
filter_test = df_test['INTENT_NAME'].isin(allowed_intents)
df_test_ = df_test[filter_test]
print(allowed_intents)

In [ ]:
ohe = OneHotEncoder()
y_train = ohe.fit_transform(df_train_['INTENT_NAME'].to_numpy().reshape((-1,1))).todense()
y_test = ohe.transform(df_test_['INTENT_NAME'].to_numpy().reshape((-1,1))).todense()
X_train = df_train_['UTTERANCES']
X_test = df_test_['UTTERANCES']